In [1]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [2]:
# set semantics and compute strength of arguments
filename = "../../bags/td_qbaf.bag"
model = grad.semantics.QuadraticEnergyModel()
model.approximator = grad.algorithms.RK4(model)
model.BAG = grad.BAG(filename)
model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 9.499999999999982
Argument(name=s0, weight=0.5, strength=0.6983561366720544)
Argument(name=s1, weight=0.5, strength=0.6983561366720544)
Argument(name=s2, weight=0.5, strength=0.6983561366720544)
Argument(name=s3, weight=0.5, strength=0.6984075829197717)
Argument(name=s4, weight=0.5, strength=0.6983866309035758)
Argument(name=s5, weight=0.5, strength=0.6983866309035758)
Argument(name=s6, weight=0.5, strength=0.5000006666489577)
Argument(name=s7, weight=0.5, strength=0.7211368661875142)
Argument(name=s8, weight=0.5, strength=0.7211368661875142)
Argument(name=s9, weight=0.5, strength=0.7211368661875142)
Argument(name=s10, weight=0.5, strength=0.7211368661875142)
Argument(name=c0, weight=0.0, strength=0.8135102276868453)
Argument(name=c1, weight=0.0, strength=7.654312473156645e-06)
Argument(name=c2, weight=0.0, strength=0.8135536884131115)
Argument(name=c3, weight=0.0, strength=6.897036405265413e-06)
Argument(name=c4, weight=0.0, strength=5.320066899352037e-

0.0009670739847200497

In [3]:
# get all the attacks and supports in QBAF
att_relation = model.BAG.get_attacks()
sup_relation = model.BAG.get_supports()
all_relation = att_relation + sup_relation

In [4]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))
all_relation_new

[('att', 'c0', 'c1'),
 ('att', 'c1', 'c0'),
 ('att', 'c2', 'c3'),
 ('att', 'c3', 'c2'),
 ('att', 'c4', 'c5'),
 ('att', 'c5', 'c4'),
 ('sup', 's0', 'c0'),
 ('sup', 's1', 'c0'),
 ('sup', 's2', 'c0'),
 ('sup', 's3', 'c1'),
 ('sup', 's3', 'c2'),
 ('sup', 's4', 'c2'),
 ('sup', 's5', 'c2'),
 ('sup', 's6', 'c3'),
 ('sup', 's6', 'c4'),
 ('sup', 's7', 'c5'),
 ('sup', 's8', 'c5'),
 ('sup', 's9', 'c5'),
 ('sup', 's10', 'c5'),
 ('sup', 'c0', 's0'),
 ('sup', 'c0', 's1'),
 ('sup', 'c0', 's2'),
 ('sup', 'c1', 's3'),
 ('sup', 'c2', 's3'),
 ('sup', 'c2', 's4'),
 ('sup', 'c2', 's5'),
 ('sup', 'c3', 's6'),
 ('sup', 'c4', 's6'),
 ('sup', 'c5', 's7'),
 ('sup', 'c5', 's8'),
 ('sup', 'c5', 's9'),
 ('sup', 'c5', 's10')]

In [5]:
# set topic argument
topic_a = "c5"
topic_arg = model.BAG.arguments[topic_a]

In [6]:
# compute strength for all arguments
for arg in model.BAG.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('s0', 0.5, 0.6983561366720544)
('s1', 0.5, 0.6983561366720544)
('s2', 0.5, 0.6983561366720544)
('s3', 0.5, 0.6984075829197717)
('s4', 0.5, 0.6983866309035758)
('s5', 0.5, 0.6983866309035758)
('s6', 0.5, 0.5000006666489577)
('s7', 0.5, 0.7211368661875142)
('s8', 0.5, 0.7211368661875142)
('s9', 0.5, 0.7211368661875142)
('s10', 0.5, 0.7211368661875142)
('c0', 0.0, 0.8135102276868453)
('c1', 0.0, 7.654312473156645e-06)
('c2', 0.0, 0.8135536884131115)
('c3', 0.0, 6.897036405265413e-06)
('c4', 0.0, 5.320066899352037e-06)
('c5', 0.0, 0.8922323980695945)


In [7]:
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations

for i in range(len(all_relation_new)):

    cur_relation = all_relation_new[i]


    # compute strength before removing
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
    topic_arg = model.BAG.arguments[topic_a]
    strength_before_drop_cur = topic_arg.strength


    # remove cur_relation
    if cur_relation[0]=="att":
        temp_attacker = model.BAG.remove_attack(cur_relation[1],cur_relation[2]) # cur_relation[1] attacks cur_relation[2]
    elif cur_relation[0]=="sup":
        temp_supporter = model.BAG.remove_support(cur_relation[1],cur_relation[2]) # cur_relation[1] supports cur_relation[2]


    # compute strength after removing
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)
    topic_arg = model.BAG.arguments[topic_a]
    strength_after_drop_cur = topic_arg.strength

    # compute strength difference
    difference = strength_before_drop_cur - strength_after_drop_cur


    # compute attributions
    attribution[cur_relation] = difference

    # recover QBAF for the next iteration
    model = grad.semantics.QuadraticEnergyModel()
    model.approximator = grad.algorithms.RK4(model)
    model.BAG = grad.BAG(filename)
    model.solve(delta=10e-2, epsilon=10e-4, verbose=True, generate_plot=False)

QuadraticEnergyModel, RK4
Time: 0.1
Argument(name=s0, weight=0.5, strength=0.6984302509001103)
Argument(name=s1, weight=0.5, strength=0.6984302509001103)
Argument(name=s2, weight=0.5, strength=0.6984302509001103)
Argument(name=s3, weight=0.5, strength=0.6984781858076413)
Argument(name=s4, weight=0.5, strength=0.6984590236274929)
Argument(name=s5, weight=0.5, strength=0.6984590236274929)
Argument(name=s6, weight=0.5, strength=0.5000006032154023)
Argument(name=s7, weight=0.5, strength=0.7211830627440531)
Argument(name=s8, weight=0.5, strength=0.7211830627440531)
Argument(name=s9, weight=0.5, strength=0.7211830627440531)
Argument(name=s10, weight=0.5, strength=0.7211830627440531)
Argument(name=c0, weight=0.0, strength=0.8136008932142184)
Argument(name=c1, weight=0.0, strength=6.925908962429875e-06)
Argument(name=c2, weight=0.0, strength=0.8136417222705967)
Argument(name=c3, weight=0.0, strength=6.240697178349343e-06)
Argument(name=c4, weight=0.0, strength=4.813796033042449e-06)
Argument(n

In [8]:
attribution

{('att', 'c0', 'c1'): -0.000606792614763707,
 ('att', 'c1', 'c0'): -4.284814013377769e-05,
 ('att', 'c2', 'c3'): -0.0006037620299091939,
 ('att', 'c3', 'c2'): -4.284814013377769e-05,
 ('att', 'c4', 'c5'): -4.287706181171913e-05,
 ('att', 'c5', 'c4'): 0.01604691751221765,
 ('sup', 's0', 'c0'): -0.0006065768202676569,
 ('sup', 's1', 'c0'): -0.0006065768202676569,
 ('sup', 's2', 'c0'): -0.0006065768202676569,
 ('sup', 's3', 'c1'): -4.284814013377769e-05,
 ('sup', 's3', 'c2'): -0.0006061086157452822,
 ('sup', 's4', 'c2'): -0.0006061086157452822,
 ('sup', 's5', 'c2'): -0.0006061086157452822,
 ('sup', 's6', 'c3'): -4.284814013377769e-05,
 ('sup', 's6', 'c4'): -4.284814013377769e-05,
 ('sup', 's7', 'c5'): 0.07583002119182058,
 ('sup', 's8', 'c5'): 0.07583002119182058,
 ('sup', 's9', 'c5'): 0.07583002119182058,
 ('sup', 's10', 'c5'): 0.07583002119182058,
 ('sup', 'c0', 's0'): -0.000599247892505983,
 ('sup', 'c0', 's1'): -0.000599247892505983,
 ('sup', 'c0', 's2'): -0.000599247892505983,
 ('sup